In [3]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from kdd_oh_set import ohkdd
import net_hook

In [4]:
use_gpu = False
train_data = ohkdd(use_gpu=use_gpu)
dataloader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
print(len(train_data))

1124263


In [5]:
color={'normal':'b','abnormal':'r','dos':'r','probe':'m','r2l':'g','u2r':'y','marker':'.'}

def fig2d(net, hook, dis, num):  #模型，钩子，距离函数
    plt.figure("aenet")
    for i, (x, y) in enumerate(dataloader):
        if i == num:
            break
        x = torch.autograd.Variable(x)
        print(x.size())
        output = net(x)
        cossim = dis(x, output).detach().numpy()
        fea = hook.fea.detach().numpy()
        for j in range(0, len(x)):
            if y[j] == 'normal':
                plt.plot(fea[j], cossim[j], color['normal']+color['marker'])
            elif y[j]=='dos':
                plt.plot(fea[j], cossim[j], color['dos']+color['marker'])
            elif y[j]=='probe':
                plt.plot(fea[j], cossim[j], color['probe']+color['marker'])
            elif y[j]=='r2l':
                plt.plot(fea[j], cossim[j], color['r2l']+color['marker'])
            elif y[j]=='u2r':
                plt.plot(fea[j], cossim[j], color['u2r']+color['marker'])
    plt.xlabel("reduce")
    plt.ylabel("distance")

    # plt.plot([], [], "b.", label='normal')
    # plt.plot([], [], "r.", label='abnormal')
    # plt.legend()

    plt.savefig("./{}_1key.svg".format(dis), format="svg")
    plt.show()


def fig3d(net, hook, dis, num):  #模型，钩子，距离函数
    ax = plt.axes(projection='3d')
    for i, (x, y) in enumerate(dataloader):
        if i == num:
            break
        x = torch.autograd.Variable(x)
        output = net(x)
        cossim = dis(x, output).detach().numpy()
        fea = hook.fea.detach().numpy()
        for j in range(0, len(x)):
            if y[j] == 'normal':
                ax.scatter(fea[j][0], fea[j][1], cossim[j], c=color['normal'], marker=color['marker'])
            if y[j] == 'dos':
                ax.scatter(fea[j][0], fea[j][1], cossim[j], c=color['dos'], marker=color['marker'])
            if y[j] == 'probe':
                ax.scatter(fea[j][0], fea[j][1], cossim[j], c=color['probe'], marker=color['marker'])
            if y[j] == 'r2l':
                ax.scatter(fea[j][0], fea[j][1], cossim[j], c=color['r2l'], marker=color['marker'])
            if y[j] == 'u2r':
                ax.scatter(fea[j][0], fea[j][1], cossim[j], c=color['u2r'], marker=color['marker'])

    ax.set_xlabel("key1")
    ax.set_ylabel("key2")
    ax.set_zlabel("dis")
    ax.set_title("ae_2key")

    # ax.scatter([], [], [], c='b', marker='.', label='normal')
    # ax.scatter([], [], [], c='r', marker='.', label='abnormal')
    # ax.legend()

    plt.savefig("./{}_2key.svg".format(dis), format="svg")
    plt.show()


def plt_from_net(net_path: str, dis, demension, num=50):
    net = torch.load(net_path)
    hook = net_hook.get_fea_by_hook(net.keylayer())
    if not use_gpu:
        net = net.cpu()
    if demension == 2:
        fig2d(net, hook, dis, num)
    elif demension == 3:
        fig3d(net, hook, dis, num)
    else:
        raise UserWarning("demension should in [2,3]")

In [6]:
plt_from_net("aenet_1key", torch.nn.CosineSimilarity(), 2, 20)
# plt_from_net("aenet_2key", torch.nn.CosineSimilarity(), 3, 5)

torch.Size([32, 1, 126])


RuntimeError: running_mean should contain 1 elements not 100

<Figure size 432x288 with 0 Axes>